In [ ]:
# Import the dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import json
import gmaps
import os

# Setting the api key
from api_keys import g_key

In [ ]:
# Create a new dataframe called vacation
file = "../Weatherpy/output/weatherpy.csv"
vacation = pd.read_csv(file)
vacation.head()

In [ ]:
# Configure the api key
gmaps.configure(api_key=g_key)

In [ ]:
# Finding the location of each city and setting the humidity value.
locations = vacation[["Latitude", "Longitude"]].astype(float)
humidity = vacation['Humidity']

In [ ]:
# Creating a figure for gmaps
fig = gmaps.figure()

# Create a heat layer for the gmap with the weight being humidity.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=vacation['Humidity'].max(),
                                 point_radius = 2)

# Adds the heat layer.
fig.add_layer(heat_layer)

# Shows the figure.
fig

In [ ]:
# Adds a new column in vacation dataframe.
vacation["Hotel Name"]= ""

# To find the comfortable vacation conditions 
hotel_df = vacation.loc[(vacation['Temperature'] > 70)&
                        (vacation['Temperature'] < 85)&
                        (vacation['Cloudiness'] == 0)&
                        (vacation['Wind Speed'] < 5)]
hotel_df

In [ ]:
# Creating a new url to find hotels within 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
radius = 5000

# Creating parameters to find specifically hotels.
param = {
    "radius" : radius,
    "types" : "lodgings",
    "key" : g_key
}

# To go through all the rows and pull the coordinates.
for index, rows in hotel_df.iterrows():
    lat = rows['Latitude']
    lng = rows['Longitude']

# Adding a new parameter, location
    param["location"] = f'{lat},{lng}'

# Creating an api request
    hotels = requests.get(base_url,params = param)
    hotels = hotels.json()

# Will append the hotel name and if there does not exist a hotel, it will skip.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels['results'][1]['name']
    except(KeyError, IndexError):
        print('Hotel not found')
    

In [ ]:
# If there are empty values(Hotels not in vacinity)
# removes the value
hotel_df = hotel_df.replace(r'^\s*$', np.NaN, regex=True)
hotel_df = hotel_df.dropna(how = 'any')
hotel_df

In [ ]:
# Creates a template for the hotel markers
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# For loop to run through the dataframe.
# Pulls information and uses the template above.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Location of each hotel
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# New figure
fig = gmaps.figure()

# New layer called hotel
# Add it to the figure
hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(hotel_layer)

fig

In [ ]:
# Create a new figure and combine both layers.
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig